In [2]:
!pip install chromadb


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()
import os, warnings, chromadb
warnings.filterwarnings('ignore')
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

In [5]:
api_key = os.environ['OPENAI_API_KEY']

In [16]:
# Load documents --> Split into chunks
loader = TextLoader(file_path="../langchain-course-main/01-Data-Connections/some_data/FDR_State_of_Union_1944.txt")
data = loader.load()

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000)
docs = text_splitter.split_documents(data)

# Embedding model -> Embed chunks -> Vectors
embeddings = OpenAIEmbeddings(api_key=api_key)

# Vectors -> Save ->  Vector store (ChromaDB)
db = Chroma.from_documents(embedding=embeddings,documents=docs,persist_directory="./speech_db")
db.persist()

In [28]:
# Chroma text str | [1,2,3]
# new_str --> [345] --> similarity search
db_new_connection = Chroma(persist_directory="./speech_db/",embedding_function=embeddings)

In [30]:
new_doc = "What did FDR say about the cost of food law?"
similar_docs = db_new_connection.similarity_search(query=new_doc,k=5)
print(similar_docs)

[Document(metadata={'source': '../langchain-course-main/01-Data-Connections/some_data/FDR_State_of_Union_1944.txt'}, page_content='If ever there was a time to subordinate individual or group selfishness to the national good, that time is now. Disunity at home—bickerings, self-seeking partisanship, stoppages of work, inflation, business as usual, politics as usual, luxury as usual these are the influences which can undermine the morale of the brave men ready to die at the front for us here.\n\nThose who are doing most of the complaining are not deliberately striving to sabotage the national war effort. They are laboring under the delusion that the time is past when we must make prodigious sacrifices- that the war is already won and we can begin to slacken off. But the dangerous folly of that point of view can be measured by the distance that separates our troops from their ultimate objectives in Berlin and Tokyo—and by the sum of all the perils that lie along the way.\n\nOverconfidence 

In [32]:
similar_docs[0].page_content

'If ever there was a time to subordinate individual or group selfishness to the national good, that time is now. Disunity at home—bickerings, self-seeking partisanship, stoppages of work, inflation, business as usual, politics as usual, luxury as usual these are the influences which can undermine the morale of the brave men ready to die at the front for us here.\n\nThose who are doing most of the complaining are not deliberately striving to sabotage the national war effort. They are laboring under the delusion that the time is past when we must make prodigious sacrifices- that the war is already won and we can begin to slacken off. But the dangerous folly of that point of view can be measured by the distance that separates our troops from their ultimate objectives in Berlin and Tokyo—and by the sum of all the perils that lie along the way.\n\nOverconfidence and complacency are among our deadliest enemies. Last spring—after notable victories at Stalingrad and in Tunisia and against the 

In [38]:
loader = TextLoader(file_path="../langchain-course-main/01-Data-Connections/some_data/Lincoln_State_of_Union_1862.txt")
loader

In [40]:
lincoln_data = loader.load()

In [42]:
docs = text_splitter.split_documents(lincoln_data)

In [44]:
db_new_connection = Chroma.from_documents(documents=docs,embedding=embeddings,persist_directory="./speech_db/")

In [46]:
docs = db_new_connection.similarity_search("slavery")

In [48]:
docs[0].page_content

'As to the first article, the main points are, first, the emancipation; secondly, the length of time for consummating it (thirty-seven years); and, thirdly, the compensation.\n\nThe emancipation will be unsatisfactory to the advocates of perpetual slavery, but the length of time should greatly mitigate their dissatisfaction. The time spares both races from the evils of sudden derangement—in fact, from the necessity of any derangement—while most of those whose habitual course of thought will be disturbed by the measure will have passed away before its consummation. They will never see it. Another class will hail the prospect of emancipation, but will deprecate the length of time. They will feel that it gives too little to the now living slaves. But it really gives them much. It saves them from the vagrant destitution which must largely attend immediate emancipation in localities where their numbers are very great, and it gives the inspiring assurance that their posterity shall be free f

In [50]:
docs[0].metadata

{'source': '../langchain-course-main/01-Data-Connections/some_data/Lincoln_State_of_Union_1862.txt'}

In [52]:
type(db_new_connection)

langchain_community.vectorstores.chroma.Chroma

In [54]:
retriever = db_new_connection.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x15c4e3f80>, search_kwargs={})

In [56]:
results = retriever.get_relevant_documents("cost of food law")
results

[Document(metadata={'source': '../langchain-course-main/01-Data-Connections/some_data/FDR_State_of_Union_1944.txt'}, page_content='If ever there was a time to subordinate individual or group selfishness to the national good, that time is now. Disunity at home—bickerings, self-seeking partisanship, stoppages of work, inflation, business as usual, politics as usual, luxury as usual these are the influences which can undermine the morale of the brave men ready to die at the front for us here.\n\nThose who are doing most of the complaining are not deliberately striving to sabotage the national war effort. They are laboring under the delusion that the time is past when we must make prodigious sacrifices- that the war is already won and we can begin to slacken off. But the dangerous folly of that point of view can be measured by the distance that separates our troops from their ultimate objectives in Berlin and Tokyo—and by the sum of all the perils that lie along the way.\n\nOverconfidence 